In [ ]:
!pip install deap
!pip install finlab >log.txt
!pip install ta-lib-bin >log.txt

import multiprocessing
from deap import base, creator, tools
import random
import functools
from finlab import data
from finlab.backtest import sim
import pandas as pd
from finlab import data
import math
from finlab.dataframe import FinlabDataFrame
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.1 MB/s eta 0:00:00


In [ ]:


# 假設這些是您的15個條件
close = data.get('price:收盤價')
rev_yoy_growth = data.get('monthly_revenue:去年同月增減(%)')
rev = data.get('monthly_revenue:當月營收')
roe = data.get('fundamental_features:ROE稅後')
volume = data.get('price:成交股數').average(5)
rev_ma2=rev.average(2)
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
rev_ma24=rev.average(24)
營業利益成長率=data.get('fundamental_features:營業利益成長率')
pe = data.get('price_earning_ratio:本益比')
peg=(pe/營業利益成長率)
rsv = (close - close.rolling(120).min()) / (close.rolling(120).max() - close.rolling(120).min())
pb_ratio = data.get('price_earning_ratio:股價淨值比')
boss_hold = data.get("internal_equity_changes:董監持有股數占比")
# 低波動因子
融資使用率 = data.get('margin_transactions:融資使用率').fillna(0)
業外收支營收率 = data.get('fundamental_features:業外收支營收率')
# 票面分割還原比例
股本 = data.get('financial_statement:股本')
empty_df = pd.DataFrame(1, columns=close.columns, index=close.index)
tse_par = data.get('par_value_change_tse:twse_par_value_change_divide_ratio')
otc_par = data.get('par_value_change_otc:otc_par_value_change_divide_ratio')
tse_divide_ratio = (empty_df*tse_par).fillna(1).cumprod()
otc_divide_ratio = (empty_df*otc_par).fillna(1).cumprod()
par_divide_ratio = tse_divide_ratio*otc_divide_ratio
市值 = 股本 * close / 10 * 1000 * par_divide_ratio
當月營收 = data.get("monthly_revenue:當月營收")
當季營收 = 當月營收.rolling(4).sum()*1000
市值營收比 = 市值 / 當季營收
#
df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()
rev_sl_growth = 當月營收.average(3).pct_change(12) > 當月營收.average(12).pct_change(12)
# 進場訊號波動率
atr = data.indicator('ATR', adjust_price=True,timeperiod=10)
adj_close = data.get('etl:adj_close')
entry_volatility = atr/adj_close
# 低波動因子
業外收支營收率 = data.get('fundamental_features:業外收支營收率')
tree_select_factor =(entry_volatility <= 0.075) & (業外收支營收率<5)
#
def compute_candle_volatility(timeperiod=20):
    close = data.get("price:收盤價")
    high = data.get("price:最高價")
    low = data.get("price:最低價")
    open_ =  data.get("price:開盤價")

    bullish_candle = close >= open_
    bullish_volatility = abs(close.shift() - open_) + abs(open_ - low) + abs(low - high) + abs(high - close)
    bearish_volatility = abs(close.shift() - open_) + abs(open_ - high) + abs(high-low) + abs(low - close)
    candle_volatility = FinlabDataFrame(np.nan, index=close.index, columns=close.columns)
    candle_volatility[bullish_candle] = bullish_volatility
    candle_volatility[~bullish_candle] = bearish_volatility
    volatility = candle_volatility.average(timeperiod) / close.average(timeperiod) * 100
    return volatility
volatility = compute_candle_volatility()


close =  data.get('price:收盤價')
volume = data.get('price:成交股數')
month_rev_growth = data.get('monthly_revenue:去年同月增減(%)')
volumereduce = (volume.average(10) < (volume.average(60)*0.5))#.sustain(4,1)
pricecontract = (close.rolling(15).std() < (close.rolling(60).std()*0.6))#.sustain(4,1)

#計算研發費用率排名
研究發展費 = data.get("financial_statement:研究發展費").deadline()
營業收入淨額 = data.get("financial_statement:營業收入淨額").deadline()
research_ratio= 研究發展費 / 營業收入淨額 * 100
research_ratio_drop = research_ratio[研究發展費 < 營業收入淨額]
research_ratio_drop = research_ratio_drop.dropna(how='all',axis=1).replace(np.inf,np.nan)
research_ratio_rank = research_ratio_drop.rank(pct=True,axis=1)

std = close.pct_change().rolling(15).std()

operating_margin_rate = data.get('fundamental_features:營業利益率')

mon_rev =data.get("monthly_revenue:當月營收")
mon_rev_lag =data.get("monthly_revenue:去年當月營收")
three_mon_rev_yoy = mon_rev.rolling(3).sum()/mon_rev_lag.rolling(3).sum()
twelve_mon_rev_yoy = mon_rev.rolling(12).sum()/mon_rev_lag.rolling(12).sum()

cond1 = rev_ma3/rev_ma12>1.1
cond2 = rev/rev.shift()>0.95
cond3 = 自由現金流>0
cond4 = rsv>0.85
cond5 = close>close.average(20)
cond6 = volatility<8
cond7 = (市值<1e10)
cond8 = (業外收支營收率<5)
cond9 = rev_sl_growth
cond10 = (close/close.rolling(100).max())==1
cond11 = ((rev.rolling(2).mean() / rev.rolling(12).mean()).index_str_to_date().rank(axis=1, pct=True).astype(float) > 0.8)
cond12 = close.rolling(20).min() > close.rolling(20).max() * 0.7
cond13 = (volumereduce & pricecontract).sustain(5, 1) & (close == close.rolling(100).max()) & (volume >= volume.rolling(20).mean()*0.8)
cond14 = (volume*close > 2000000)
cond15 = close > close.average(250)
cond16 = ((close / close.rolling(10).max())==1).sustain(4,2)
cond17 = (research_ratio_rank >= 0.8)
cond18 = 市值 > 市值.quantile_row(0.1)
cond19 = (month_rev_growth>0).sustain(2)
cond20 = (three_mon_rev_yoy/twelve_mon_rev_yoy >1.05)
cond21 = (std.rank(pct=True, axis=1) > 0.45).astype(float)
cond22 = operating_margin_rate > operating_margin_rate.quantile_row(0.6)

sell1 = (close / close.average(50) < 0.90)
sell2 = (close / close.average(20) < 0.95)
sell3 = (close < close.average(20) ).sustain(2) | close.isna().sustain(20)

輸入成功!


In [ ]:

import multiprocessing
conditions = [cond1, cond2, cond3, cond4, cond5, cond6, cond7, cond8, cond9, cond10, cond11, cond12, cond13, cond14, cond15, cond16, cond17, cond18, cond19, cond20, cond21, cond22]


sell1 = (close / close.average(50) < 0.90)
sell2 = (close / close.average(20) < 0.95)
sell3 = (close < close.average(20) ).sustain(2) | close.isna().sustain(20)
# 創建問題類型和個體類型

buy_condition_number = 22
buy_max_condition_number = 8

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)  # 二進制編碼
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=25)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    selected_conditions = [cond for cond, selected in zip(conditions, individual[:buy_condition_number]) if selected]
    buy = functools.reduce(lambda x, y: x & y, selected_conditions)

    # 确保只有一个卖出条件为真
    sell_conditions = [sell1, sell2, sell3]
    active_sell = [sell_cond for cond, sell_cond in zip(individual[buy_condition_number :], sell_conditions) if cond]

    # 如果没有有效的卖出条件，默认使用第一个
    if not active_sell:
        active_sell = [sell1]

    position = p = (buy).hold_until(active_sell[0], nstocks_limit=15, rank=((month_rev_growth / month_rev_growth.average(6)).index_str_to_date()), stop_loss=0.1)
    report = sim(position=position, fee_ratio=1.425/1000/3,upload=False, position_limit=0.2, trade_at_price='close')
    指標 = (report.get_stats()['daily_mean'] / report.get_stats()['max_drawdown']) * report.get_stats()['daily_sortino']
    # 在返回指標之前檢查它的值
    if not isinstance(指標, float) or math.isnan(指標):
        print("警告: 指標的值不正確:", 指標)
        指標 = 0.0  # 或者您可以選擇一個合適的默認值
    return (-指標,)  # 由於我們想極大化指標，所以返回負值
stats = tools.Statistics(lambda ind: ind.fitness.values[0])  # 使用第一個元素
stats.register("avg", lambda x: sum(x) / len(x))
stats.register("max", max)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)
# 在toolbox中添加統計工具
stats = tools.Statistics(lambda ind: ind.fitness.values[0] if ind.fitness.values else 0)

stats.register("avg", lambda x: sum(x) / len(x))
stats.register("max", max)
# 創建初始族群
population = toolbox.population(n=50)

# 進行遺傳算法優化
NGEN = 100  # 世代數量
CXPB = 0.75 # 交叉概率
MUTPB = 0.1  # 變異概率

# 修正過多的條件，只保留最多7個條件
def enforce_max_conditions(ind):
    true_conditions = [i for i, condition in enumerate(ind) if condition]
    while len(true_conditions) > buy_max_condition_number:
        ind[random.choice(true_conditions)] = 0
        true_conditions = [i for i, condition in enumerate(ind) if condition]

def enforce_one_sell_condition(ind):
    sell_genetic = ind[buy_condition_number :]
    true_conditions = [i for i, condition in enumerate(sell_genetic) if condition]
    while len(true_conditions) != 1:
        if len(true_conditions) > 1:
            ind[buy_condition_number  + random.choice(true_conditions)] = 0
        else:
            ind[buy_condition_number  + random.choice(range(3))] = 1
        sell_genetic = ind[buy_condition_number :]
        true_conditions = [i for i, condition in enumerate(sell_genetic) if condition]


for gen in range(NGEN):
    print(f"世代 {gen + 1}/{NGEN}:")  # 打印當前世代
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 進行交叉
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # 進行變異
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 保證條件為真的數量不超過5
    for individual in offspring:
        enforce_max_conditions(individual)

    for individual in offspring:
        enforce_one_sell_condition(individual)

    # 評估新的個體
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

    # 使用多線程平行計算適應度
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    fitnesses = pool.map(toolbox.evaluate, invalid_ind)
    pool.close()
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    # 計算並打印統計信息
    record = stats.compile(population)
    print(f"  平均適應度: {record['avg']:.3f}")
    print(f"  最大適應度: {record['max']:.3f}")

    # 替換族群
    population[:] = offspring



# 打印最佳解
best_ind = tools.selBest(population, 1)[0]
print("最佳組合:", best_ind)
條件名稱 = [
    "cond1", "cond2", "cond3", "cond4", "cond5",
    "cond6", "cond7", "cond8", "cond9", "cond10", "cond11", "cond12", "cond13", "cond14", "cond15", "cond16", "cond17", "cond18", "cond19", "cond20", "cond21", "cond22"]

最佳組合條件 = ' & '.join([名稱 for 名稱, 組合 in zip(條件名稱, best_ind) if 組合])

print("最佳組合包括以下條件:")
print(最佳組合條件)

條件名稱 += ['sell1', 'sell2', 'sell3']

best_ind = tools.selBest(population, 1)[0]
print("最佳賣出組合:", best_ind)
條件名稱 = [
"sell1", "sell2", "sell3"]

最佳賣出組合條件 = ' & '.join([名稱 for 名稱, 組合 in zip(條件名稱, best_ind[buy_condition_number :] ) if 組合])

print("最佳賣出組合包括以下條件:")
print(最佳賣出組合條件)

buyy = eval(最佳組合條件)
selll = eval(最佳賣出組合條件)

position = p = (buyy).hold_until(((selll)), nstocks_limit = 15, rank = ((month_rev_growth / month_rev_growth.average(6)).index_str_to_date()), stop_loss = 0.1)

report = sim(position=position, fee_ratio=1.425/1000/3,upload=False, position_limit=0.3, trade_at_price='close')
report.display()

世代 1/100:


  平均適應度: 0.000
  最大適應度: 0.000
世代 2/100:


  平均適應度: 1.949
  最大適應度: 6.757
世代 3/100:


  平均適應度: 2.236
  最大適應度: 3.862
世代 4/100:


  平均適應度: 2.629
  最大適應度: 4.467
世代 5/100:


...

  平均適應度: 9.096
  最大適應度: 9.986
世代 99/100:


  平均適應度: 9.855
  最大適應度: 9.986
世代 100/100:


  平均適應度: 9.357
  最大適應度: 9.986
最佳組合: [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
最佳組合包括以下條件:
cond2 & cond6 & cond7 & cond9 & cond11 & cond13 & cond14
最佳賣出組合: [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
最佳賣出組合包括以下條件:
sell2


,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,49.17%,2.13,-18.51%,53.04%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,trade_price@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,
1465 偉全,2021-07-26,2023-07-28,2021-07-23,2023-07-27,0.083333,489,3519,4008,0.126660,13.30,13.45,-0.060953,0.143413,0.023715,-0.082707,251.0,0.00,0.00
6703 軒郁,2023-07-11,2023-08-04,2023-07-10,2023-08-02,0.083333,17,3995,4012,0.101103,272.00,299.50,0.000000,0.244485,0.000000,-0.184638,17.0,0.00,0.00
6538 倉和,2023-07-06,2023-08-11,2023-07-05,2023-08-10,0.076923,25,3992,4017,0.157122,137.50,155.50,0.000000,0.220373,0.000000,-0.137195,25.0,0.00,0.00
5516 雙喜,2023-05-18,2023-08-14,2023-05-17,2023-08-11,0.076923,59,3959,4018,0.001767,28.30,28.35,-0.026502,0.166078,0.000000,-0.140909,46.0,0.00,0.00
5511 德昌,2023-07-21,2023-08-15,2023-07-20,2023-08-14,0.083333,16,4003,4019,-0.049682,78.50,74.60,-0.067516,0.091720,0.091720,-0.145858,13.0,0.00,0.00
3147 大綜,2023-06-15,2023-08-25,2023-06-14,2023-08-24,0.066667,48,3979,4027,-0.085965,79.50,68.30,-0.099371,0.000000,0.000000,-0.099371,0.0,0.00,0.00
1784 訊聯,2023-07-18,2023-08-29,2023-07-17,2023-08-28,0.090909,29,4000,4029,-0.052553,66.60,63.10,-0.058559,0.096096,0.096096,-0.141096,12.0,0.00,0.00
3088 艾訊,2023-07-13,2023-08-29,2023-07-12,2023-08-28,0.100000,32,3997,4029,0.053937,92.70,97.70,0.000000,0.192017,0.000000,-0.126697,32.0,0.00,0.00
2063 世鎧,2023-07-04,NaT,2023-07-03,NaT,0.076923,42,3990,-1,-0.070209,52.70,49.00,-0.096774,0.005693,0.005693,-0.101887,1.0,0.25,0.25


In [ ]:


buyy = cond1 & cond3 & cond7 & cond8 & cond10 & cond16 & cond22
selll = sell3
position = p = (buyy).hold_until(((selll)), nstocks_limit = 15, rank = ((month_rev_growth / month_rev_growth.average(6)).index_str_to_date()), stop_loss = 0.1)

report = sim(position=position, fee_ratio=1.425/1000/3,upload=True, position_limit=0.3, trade_at_price='close',name = "基因魔龍")
report.display()

,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,60.34%,2.25,-18.75%,46.37%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,trade_price@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,
4561 健椿,2023-08-09,2023-08-25,2023-08-08,2023-08-24,0.076923,12.0,4015.0,4027.0,-0.027624,27.15,26.40,-0.033149,1.841621e-03,0.001842,-0.034926,3.0,0.000000,0.000000
4572 駐龍,2023-08-01,2023-08-28,2023-07-31,2023-08-25,0.066667,18.0,4010.0,4028.0,0.055638,196.00,188.00,-0.058673,6.686809e-02,0.007653,-0.065823,6.0,0.000000,0.000000
5432 新門,2023-07-19,2023-08-28,2023-07-18,2023-08-25,0.083333,27.0,4001.0,4028.0,0.233333,39.00,48.10,-0.057692,3.307692e-01,0.000000,-0.146146,25.0,0.000000,0.000000
3583 辛耘,2023-05-29,2023-08-29,2023-05-26,2023-08-28,0.066667,63.0,3966.0,4029.0,1.133618,103.00,215.00,0.000000,1.550418e+00,0.000000,-0.184369,63.0,0.000000,0.000000
2488 漢平,2023-08-11,2023-08-30,2023-08-10,2023-08-29,0.066667,13.0,4017.0,4030.0,0.023481,36.20,37.05,-0.005525,6.353591e-02,0.000000,-0.062338,11.0,0.000000,0.000000
6449 鈺邦,2023-07-24,2023-08-30,2023-07-21,2023-08-29,0.083333,26.0,4004.0,4030.0,-0.010955,63.90,63.20,-0.021909,9.389671e-02,0.028169,-0.103004,16.0,0.000000,0.000000
3043 科風,2023-08-17,NaT,2023-08-16,NaT,0.076923,11.0,4021.0,-1.0,0.263462,52.00,65.70,0.000000,2.634615e-01,0.000000,-0.023091,10.0,0.166667,0.142857
3567 逸昌,2023-07-25,NaT,2023-07-24,NaT,0.083333,27.0,4005.0,-1.0,0.161383,34.70,40.30,-0.040346,2.593660e-01,0.000000,-0.102975,15.0,0.166667,0.142857
5236 凌陽創新,2023-08-24,NaT,2023-08-23,NaT,0.083333,6.0,4026.0,-1.0,-0.007273,137.50,136.50,-0.047273,2.220446e-16,0.000000,-0.047273,1.0,0.166667,0.142857
